In [1]:
import os
import re
import pandas as pd
import numpy as np
import tensorflow as tf

/home/user/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/user/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/user/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/user/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

### Load Data

In [2]:
df_train = pd.read_csv('../lstm/datafiles/train.csv')
df_train.head()

,filename,text,label
0,D36051.pdf.out.html.txt,janet ley approval sow mcw dba dmi mobility s...,3
1,D07271.pdf.out.html.txt,agreement this schedule a this schedule is att...,1
2,D28723.pdf.out.html.txt,wolters kluwer contingent staffing request fo...,3
3,D42247.pdf.out.html.txt,agreement received this agreement is entered i...,2
4,D19377.pdf.out.html.txt,addendum to hosting and services agreement thi...,0


In [3]:
df_train.shape

(15505, 3)

In [4]:
df_test = pd.read_csv('../lstm/datafiles/test.csv')
df_test.head()

,text,label
0,addendum this software maintenance support and...,1
1,cc h statement of work schedule a financial s...,3
2,wolters kluwer certification the undersigned ...,3
3,wolters kluwer westpoint statement of work fo...,3
4,wolters kluwer contingent staffing request fo...,3


In [5]:
df_test.shape

(294, 2)

In [6]:
df_train_filtered = df_train.text.tolist()

In [7]:
y_train = df_train.label.tolist()

In [8]:
df_train_latent = pd.read_csv('../xgboost/datafiles/latent_features_train.csv')
df_train_latent.head()

,DocID,addendum date,addendum effective date,addendum entered,addendum made,addendum number,addendum_keywords_count,addendum_regex_lookup,affiliated entity,agreement hereby amended,...,sow shall,sow term,sow_keywords_count,sow_regex_lookup,statement of work effective,subject matter hereof,subsequent addendum,term of addendum,term of amendment,work sow
0,D36051.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,D07271.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,D28723.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,D42247.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,D19377.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df_train_latent = df_train_latent.drop(['DocID'], axis=1)
df_train_latent.shape

(15505, 64)

In [10]:
df_test_latent = pd.read_csv('../xgboost/datafiles/latent_features_test.csv')
df_test_latent.head()

,DocID,addendum date,addendum effective date,addendum entered,addendum made,addendum number,addendum_keywords_count,addendum_regex_lookup,affiliated entity,agreement hereby amended,...,sow shall,sow term,sow_keywords_count,sow_regex_lookup,statement of work effective,subject matter hereof,subsequent addendum,term of addendum,term of amendment,work sow
0,D00152.pdf.out.html.txt,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,D16833.pdf.out.html.txt,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,D08368.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,D38435.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,D00918.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df_test_latent = df_test_latent.drop(['DocID'], axis=1)
df_test_latent.shape

(1364, 64)

In [12]:
df_labeled = pd.read_csv('../csv/labeled_data_relabeled.csv')
df_labeled.head()

,filename,text,label
0,D00152.pdf.out.html.txt,addendum no to master services agreement this ...,Addendum
1,D16833.pdf.out.html.txt,addendum no to master services agreement this ...,Addendum
2,D08368.pdf.out.html.txt,agreement i to add effective as of january the...,Addendum
3,D38435.pdf.out.html.txt,amendment number one to work order this amendm...,Addendum
4,D00918.pdf.out.html.txt,agreement and or enrollment identified above t...,Addendum


In [13]:
df_labeled.shape

(1364, 3)

### Training

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 512 # cut texts after this number of words
latent_features_size = df_train_latent.shape[1] 

# prepare tokenizer
t = Tokenizer(num_words=max_features)
t.fit_on_texts(df_train_filtered)
#t.fit_on_texts(df_valid)
#post_seq_train = t.texts_to_sequences(df_train_filtered)
#post_seq_valid = t.texts_to_sequences(df_valid)
#X_train = pad_sequences(post_seq_train, maxlen=max_features, padding='post')
#X_valid = pad_sequences(post_seq_valid, maxlen=max_features, padding='post')

Using TensorFlow backend.


In [15]:
#print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)
#print(X_train.shape, len(y_train))

In [16]:
word_index = t.word_index
len(word_index)

386003

In [17]:
EMBEDDING_DIM = 128

In [ ]:
'''from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pickle

vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=512)
X_train = transformer.fit_transform(vectorizer.fit_transform(df_train_filtered))

#Save vectorizer.vocabulary_
pickle.dump(vectorizer.vocabulary_,open("count_vectorizer.pkl","wb"))'''

#X_dev = vectorizer.transform(df_dev.text.tolist())
#X_valid = vectorizer.transform(df_valid)
#X_test = vectorizer.transform(df_labeled.text.tolist())

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle
vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=512)
#vectorizer.fit(df_train_filtered)
#pickle.dump(vectorizer,open("count_vectorizer1.pkl","wb"))

X_train = vectorizer.fit_transform(df_train_filtered)

In [19]:
#vectorizer1 = pickle.load(open("count_vectorizer1.pkl", "rb"))
X_test = vectorizer.transform(df_labeled.text.tolist())

In [20]:
from keras.engine import Layer, InputSpec
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Embedding, Conv1D, Activation, ZeroPadding1D, Permute, Reshape, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate

class KMaxPooling(Layer):
    """
    K-max pooling layer that extracts the k-highest activations from a sequence (2nd dimension).
    TensorFlow backend.
    """
    def __init__(self, k=1, **kwargs):
        super().__init__(**kwargs)
        self.input_spec = InputSpec(ndim=3)
        self.k = k

    def compute_output_shape(self, input_shape):
        return (input_shape[0], (input_shape[1] * self.k))

    def call(self, inputs):
        
        # swap last two dimensions since top_k will be applied along the last dimension
        #shifted_input = tf.transpose(inputs, [0, 2, 1])
        
        # extract top_k, returns two tensors [values, indices]
        top_k = tf.nn.top_k(inputs, k=self.k, sorted=True, name=None)[0]
        
        # return flattened output
        return top_k

In [21]:
# two kinds of k's and kernel sizes for each operation
def two_conv_dynamic_cnn(k1 = 12, k2 = 8, ksize1 = 5, ksize2 = 5):
    inputs = Input(shape=(max_features,))
    #inputs_latent = Input(shape=(latent_features_size,))
    embed = Embedding(len(word_index), 128, input_length=512)(inputs)
    conv_results = []
    # two feature maps using for loop
    for i in range(2):
        padded = ZeroPadding1D(ksize1 - 1)(embed)
        conv1 = Conv1D(EMBEDDING_DIM, ksize1, activation = 'relu')(padded)
        permuted = Permute((2,1))(conv1)
        kmaxpool1 = KMaxPooling(k1)(permuted)
        kmaxpool1 = Reshape((k1, -1))(kmaxpool1)
        padded = ZeroPadding1D(ksize2 -1)(kmaxpool1)
        conv2 = Conv1D(EMBEDDING_DIM, ksize2, activation = 'relu')(padded)
        permuted = Permute((2,1))(conv2)
        kmaxpool2 = KMaxPooling(k2)(permuted)
        kmaxpool2 = Reshape((k2, -1))(kmaxpool2)
        flattened = Flatten()(kmaxpool2)
        conv_results.append(flattened)
    x = concatenate(conv_results)
    #x = concatenate([x, inputs_latent], axis=1)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(5, activation='softmax')(x)
    
    #model = Model(inputs = [inputs, inputs_latent], outputs = outputs)
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [22]:
two_conv_dynamic_cnn = two_conv_dynamic_cnn()
two_conv_dynamic_cnn.summary()




Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
keep_dims is deprecated, use keepdims instead

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 512)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 512, 128)      49408384    input_1[0][0]                    
____________________________________________________________________________________________________
zero_padding1d_1 (ZeroPadding1D) (None, 520, 128)      0           embedding_1[0][0]                
___________________________

In [23]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
lr = 3e-4
epochs = 10
#decay=lr/epochs
adam = Adam(lr=lr)
batch_size = 32

two_conv_dynamic_cnn.compile(adam, 'categorical_crossentropy', metrics=['accuracy'])

In [24]:
model_chkpoint = ModelCheckpoint(filepath='best_model.hdf5', save_best_only=True, save_weights_only=True)

In [25]:
reduce_lr = ReduceLROnPlateau(monitor='val_acc', 
                                patience=3, 
                                verbose=1, 
                                factor=0.2, 
                                min_lr=1e-7)

In [26]:
import pandas as pd
T_train = pd.get_dummies(y_train)
#T_valid = pd.get_dummies(y_valid)

In [29]:
T_train.iloc

0        0
1        0
2        0
3        0
4        1
        ..
15500    0
15501    0
15502    0
15503    0
15504    0
Name: 0, Length: 15505, dtype: uint8

In [37]:
X_train[2167].todense()
df_train_latent

matrix([[ 0,  0,  3,  1,  0,  0,  0,  3,  2,  1,  0,  0,  0,  0,  0,  0,
          1,  1,  0,  0,  0,  0,  0,  2,  0,  0,  0,  2,  2, 16,  0,  0,
          0,  0,  0,  0,  0,  4,  0,  0,  0,  0,  5,  0,  0,  0,  0,  0,
          3,  3,  0,  1,  2,  2,  1,  0,  0,  0,  0,  7,  0,  1,  0,  0,
          0,  2,  1,  0,  2,  0,  6,  0,  0,  0,  0,  0,  0,  2,  0,  0,
          0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  1, 12,  0,
          0,  4,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  2,  2,  0,  3,  0, 17,  0,  4,
          2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  2,  0,  0,  0,
          0,  0,  1,  0,  0,  0,  0, 11,  0,  1,  0,  2,  0,  2,  1,  1,
          0,  1,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          9,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  3,  0,  0,
          1,  1,  3,  0,  3,  0,  1,  0,  0,  0,  0

In [28]:
"""two_conv_dynamic_cnn.fit([X_train, df_train_latent], T_train,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True, 
          validation_split=0.05,
          callbacks=[reduce_lr, model_chkpoint])"""

'two_conv_dynamic_cnn.fit([X_train, df_train_latent], T_train,\n          batch_size=batch_size,\n          epochs=epochs,\n          shuffle=True, \n          validation_split=0.05,\n          callbacks=[reduce_lr, model_chkpoint])'

In [31]:
T_train = np.array(T_train)

In [34]:
T_train[:5]

array([[0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0]], dtype=uint8)

In [32]:
two_conv_dynamic_cnn.fit(X_train, T_train,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=False, 
          validation_split=0.05,
          callbacks=[reduce_lr, model_chkpoint])

Train on 14729 samples, validate on 776 samples
Epoch 1/10


ValueError: setting an array element with a sequence.

In [ ]:
vectorizer1

In [ ]:
vectorizer

### Predictions

In [ ]:
X_test.shape

In [35]:
two_conv_dynamic_cnn.load_weights('models/dcnn-10epochs-90.0-98.97-99.52.hdf5')

ValueError: Dimension 0 in both shapes must be equal, but are 2048 and 2112. Shapes are [2048] and [2112]. for 'Assign_9' (op: 'Assign') with input shapes: [2048], [2112].

In [ ]:
df_test_latent.shape, X_test.shape

In [ ]:
type(X_test.toarray()), type(df_test_latent)

In [ ]:
#text_pred = X_test
#t.fit_on_texts(text_pred)
#post_seq_test = t.texts_to_sequences(text_pred)
#x_test = pad_sequences(post_seq_test, maxlen=max_features, padding='post')
probs_test = two_conv_dynamic_cnn.predict([X_test, df_test_latent])

In [ ]:
labels_test = df_labeled.label.map({'Addendum': 0, 'MSA': 1, 'SOW': 4, 'NDA': 2, 'Others': 3})
len(labels_test)

In [ ]:
from snorkel.analysis import metric_score
test_acc = metric_score(golds=labels_test, preds=probs_test.argmax(axis=1), metric="accuracy")
print(f"Test Accuracy: {test_acc * 100:.1f}%")

In [ ]:
df_labeled['preds'] = pd.Series(probs_test.argmax(axis=1)).map({0: 'Addendum', 1: 'MSA', 4: 'SOW', 2: 'NDA', 3: 'Others'})

In [ ]:
df_labeled.head()

In [ ]:
df_labeled.to_csv('../predictions/dcnn_predictions_after_retrain.csv', index=False)

### Plot train and validation loss

In [ ]:
# list all data in history
print(two_conv_dynamic_cnn.history.history.keys())

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Plot history for loss
plt.plot(two_conv_dynamic_cnn.history.history['loss'])
plt.plot(two_conv_dynamic_cnn.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
df_labeled.label.value_counts()

In [ ]:
#Confusion matrix
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
 

cm  = confusion_matrix(labels_test, probs_test.argmax(axis=1))
fig = plt.figure()
plot_confusion_matrix(cm, figsize=(8,6), hide_ticks=False, cmap=plt.cm.Reds)
plt.xticks(range(5), ('Add', 'MSA', 'NDA', 'Others', 'SOW'), fontsize=12)
plt.yticks(range(5), ('Add', 'MSA', 'NDA', 'Others', 'SOW'), fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(labels_test, probs_test.argmax(axis=1), digits=3))